In [1]:
!pip install pycaret --quiet
!pip install shap --quiet
#colab 환경에서 실행

     |████████████████████████████████| 320 kB 7.3 MB/s 
     |████████████████████████████████| 6.8 MB 48.0 MB/s 
     |████████████████████████████████| 1.7 MB 38.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 167 kB 38.3 MB/s 
     |████████████████████████████████| 118 kB 46.9 MB/s 
     |████████████████████████████████| 16.8 MB 37.6 MB/s 
     |████████████████████████████████| 1.3 MB 39.7 MB/s 
     |████████████████████████████████| 2.0 MB 40.8 MB/s 
     |████████████████████████████████| 261 kB 65.0 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
     |████████████████████████████████| 88 kB 8.3 MB/s 
     |████████████████████████████████| 102 kB 10.4 MB/s 
     |████████████████████████████████| 690 kB 57.8 MB/s 
     |████████████████████████████████| 636 kB 55.1 MB/s 
     |█████████████████

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pycaret.classification import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/volleyball_fof.csv')
df.head()

,Unnamed: 0,A팀_득점,A팀_범실,A팀_공격종합성공률,A팀_오픈_성공률,A팀_속공_성공률,A팀_퀵오픈_성공률,A팀_서브_성공률,A팀_블로킹_성공률,A팀_디그_성공률,A팀_디그_점유율,A팀_리시브_성공률,A팀_리시브_점유율,A팀MVP,target,B팀_득점,B팀_범실,B팀_공격종합성공률,B팀_오픈_성공률,B팀_속공_성공률,B팀_퀵오픈_성공률,B팀_서브_성공률,B팀_블로킹_성공률,B팀_디그_성공률,B팀_디그_점유율,B팀_리시브_성공률,B팀_리시브_점유율,B팀MVP
0,0,69.0,25.0,17.985000,9.437500,2.083750,26.931250,0.175000,11.702128,85.714286,12.498750,40.404040,12.498750,0,1.0,82.0,27.0,12.320000,6.898182,5.627273,15.150909,0.127273,20.000000,86.111111,9.029091,50.588235,9.090909,1
1,1,78.0,13.0,32.868889,35.226667,2.222222,12.036667,0.122222,9.090909,81.250000,10.957778,47.572816,11.111111,0,1.0,102.0,29.0,20.770000,14.957778,13.226667,13.457778,0.111111,19.047619,88.636364,11.112222,45.192308,11.111111,1
2,2,88.0,25.0,27.587273,22.803636,15.426364,10.505455,0.163636,9.722222,83.928571,9.010000,44.329897,9.090000,0,0.0,82.0,22.0,25.871000,22.885000,14.047000,9.684000,0.140000,11.842105,80.357143,10.001000,38.775510,9.998000,0
3,3,80.0,19.0,32.482222,32.128889,22.222222,16.168889,0.155556,10.447761,84.126984,10.852222,54.255319,10.880000,0,1.0,86.0,21.0,29.013333,18.535556,16.032222,11.111111,0.066667,14.141414,87.272727,11.111111,43.010753,11.112222,1
4,4,68.0,21.0,23.441250,19.438750,12.500000,19.633750,0.075000,11.363636,88.118812,12.377500,40.860215,12.367500,1,1.0,86.0,25.0,22.623000,14.401000,5.000000,8.718000,0.173000,17.741935,88.073394,9.643000,40.000000,9.238000,0


In [5]:
#각 팀의 종합 성공률만
df1 = df.drop(df.loc[:,'A팀_오픈_성공률':'A팀_디그_성공률'],1)
df1 = df1.drop(df1.loc[:,'B팀_오픈_성공률':'B팀_디그_성공률'],1)
df1.head(5)

,Unnamed: 0,A팀_득점,A팀_범실,A팀_공격종합성공률,A팀_디그_점유율,A팀_리시브_성공률,A팀_리시브_점유율,A팀MVP,target,B팀_득점,B팀_범실,B팀_공격종합성공률,B팀_디그_점유율,B팀_리시브_성공률,B팀_리시브_점유율,B팀MVP
0,0,69.0,25.0,17.985000,12.498750,40.404040,12.498750,0,1.0,82.0,27.0,12.320000,9.029091,50.588235,9.090909,1
1,1,78.0,13.0,32.868889,10.957778,47.572816,11.111111,0,1.0,102.0,29.0,20.770000,11.112222,45.192308,11.111111,1
2,2,88.0,25.0,27.587273,9.010000,44.329897,9.090000,0,0.0,82.0,22.0,25.871000,10.001000,38.775510,9.998000,0
3,3,80.0,19.0,32.482222,10.852222,54.255319,10.880000,0,1.0,86.0,21.0,29.013333,11.111111,43.010753,11.112222,1
4,4,68.0,21.0,23.441250,12.377500,40.860215,12.367500,1,1.0,86.0,25.0,22.623000,9.643000,40.000000,9.238000,0


In [6]:
# #A팀 오픈~디그, B팀 오픈~디그
df2 = df.drop(['A팀_공격종합성공률', 'B팀_공격종합성공률'],1)
df2.head()

,Unnamed: 0,A팀_득점,A팀_범실,A팀_오픈_성공률,A팀_속공_성공률,A팀_퀵오픈_성공률,A팀_서브_성공률,A팀_블로킹_성공률,A팀_디그_성공률,A팀_디그_점유율,A팀_리시브_성공률,A팀_리시브_점유율,A팀MVP,target,B팀_득점,B팀_범실,B팀_오픈_성공률,B팀_속공_성공률,B팀_퀵오픈_성공률,B팀_서브_성공률,B팀_블로킹_성공률,B팀_디그_성공률,B팀_디그_점유율,B팀_리시브_성공률,B팀_리시브_점유율,B팀MVP
0,0,69.0,25.0,9.437500,2.083750,26.931250,0.175000,11.702128,85.714286,12.498750,40.404040,12.498750,0,1.0,82.0,27.0,6.898182,5.627273,15.150909,0.127273,20.000000,86.111111,9.029091,50.588235,9.090909,1
1,1,78.0,13.0,35.226667,2.222222,12.036667,0.122222,9.090909,81.250000,10.957778,47.572816,11.111111,0,1.0,102.0,29.0,14.957778,13.226667,13.457778,0.111111,19.047619,88.636364,11.112222,45.192308,11.111111,1
2,2,88.0,25.0,22.803636,15.426364,10.505455,0.163636,9.722222,83.928571,9.010000,44.329897,9.090000,0,0.0,82.0,22.0,22.885000,14.047000,9.684000,0.140000,11.842105,80.357143,10.001000,38.775510,9.998000,0
3,3,80.0,19.0,32.128889,22.222222,16.168889,0.155556,10.447761,84.126984,10.852222,54.255319,10.880000,0,1.0,86.0,21.0,18.535556,16.032222,11.111111,0.066667,14.141414,87.272727,11.111111,43.010753,11.112222,1
4,4,68.0,21.0,19.438750,12.500000,19.633750,0.075000,11.363636,88.118812,12.377500,40.860215,12.367500,1,1.0,86.0,25.0,14.401000,5.000000,8.718000,0.173000,17.741935,88.073394,9.643000,40.000000,9.238000,0


In [7]:
##############################df1 automl#################################3

In [8]:
SEED = 0
rg = setup(data = df1, target = 'target', session_id = SEED)

,Description,Value
0,session_id,0
1,Target,target
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(474, 16)"
5,Missing Values,False
6,Numeric Features,13
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


In [9]:
top5 = compare_models(sort='Accuracy', n_select=5 )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9578,0.9861,0.9566,0.9604,0.9577,0.9157,0.9172,0.382
ridge,Ridge Classifier,0.9517,0.0000,0.9449,0.9599,0.9509,0.9034,0.9062,0.011
lda,Linear Discriminant Analysis,0.9517,0.9847,0.9449,0.9599,0.9509,0.9034,0.9062,0.014
qda,Quadratic Discriminant Analysis,0.9398,0.9686,0.9386,0.9405,0.9387,0.8793,0.8810,0.014
ada,Ada Boost Classifier,0.9035,0.9534,0.9015,0.9061,0.9010,0.8065,0.8108,0.101
gbc,Gradient Boosting Classifier,0.9035,0.9623,0.8831,0.9221,0.8998,0.8064,0.8103,0.119
lightgbm,Light Gradient Boosting Machine,0.8854,0.9673,0.8581,0.9091,0.8801,0.7702,0.7754,0.072
knn,K Neighbors Classifier,0.8824,0.9426,0.8585,0.8990,0.8770,0.7643,0.7670,0.116
rf,Random Forest Classifier,0.8734,0.9474,0.8585,0.8853,0.8672,0.7462,0.7525,0.483
svm,SVM - Linear Kernel,0.8729,0.0000,0.9342,0.8499,0.8789,0.7479,0.7724,0.012


In [10]:
blended = blend_models(estimator_list=top5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9412,0.0,0.8824,1.0000,0.9375,0.8824,0.8885
1,0.9394,0.0,0.9412,0.9412,0.9412,0.8787,0.8787
2,1.0000,0.0,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.9697,0.0,0.9375,1.0000,0.9677,0.9392,0.9410
4,0.9394,0.0,0.9375,0.9375,0.9375,0.8787,0.8787
5,0.9394,0.0,1.0000,0.8889,0.9412,0.8791,0.8856
6,0.9091,0.0,0.8750,0.9333,0.9032,0.8177,0.8192
7,0.9697,0.0,0.9375,1.0000,0.9677,0.9392,0.9410
8,1.0000,0.0,1.0000,1.0000,1.0000,1.0000,1.0000


In [11]:
mymodels = tune_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9118,0.0,0.8824,0.9375,0.9091,0.8235,0.8250
1,0.9394,0.0,0.9412,0.9412,0.9412,0.8787,0.8787
2,1.0000,0.0,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.9697,0.0,0.9375,1.0000,0.9677,0.9392,0.9410
4,0.9697,0.0,0.9375,1.0000,0.9677,0.9392,0.9410
5,0.9394,0.0,1.0000,0.8889,0.9412,0.8791,0.8856
6,0.9091,0.0,0.8750,0.9333,0.9032,0.8177,0.8192
7,0.9697,0.0,0.9375,1.0000,0.9677,0.9392,0.9410
8,1.0000,0.0,1.0000,1.0000,1.0000,1.0000,1.0000


In [12]:
final_model = finalize_model(mymodels)

In [13]:
pred = predict_model(final_model, data=df1)
pred

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9747,0.9748,0.9784,0.97,0.9741,0.9493,0.9494


,Unnamed: 0,A팀_득점,A팀_범실,A팀_공격종합성공률,A팀_디그_점유율,A팀_리시브_성공률,A팀_리시브_점유율,A팀MVP,target,B팀_득점,B팀_범실,B팀_공격종합성공률,B팀_디그_점유율,B팀_리시브_성공률,B팀_리시브_점유율,B팀MVP,Label
0,0,69.0,25.0,17.985000,12.498750,40.404040,12.498750,0,1.0,82.0,27.0,12.320000,9.029091,50.588235,9.090909,1,1.0
1,1,78.0,13.0,32.868889,10.957778,47.572816,11.111111,0,1.0,102.0,29.0,20.770000,11.112222,45.192308,11.111111,1,1.0
2,2,88.0,25.0,27.587273,9.010000,44.329897,9.090000,0,0.0,82.0,22.0,25.871000,10.001000,38.775510,9.998000,0,0.0
3,3,80.0,19.0,32.482222,10.852222,54.255319,10.880000,0,1.0,86.0,21.0,29.013333,11.111111,43.010753,11.112222,1,1.0
4,4,68.0,21.0,23.441250,12.377500,40.860215,12.367500,1,1.0,86.0,25.0,22.623000,9.643000,40.000000,9.238000,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,469,46.0,10.0,34.716000,17.500000,50.943396,19.274000,0,0.0,44.0,17.0,33.135000,12.286250,31.250000,12.500000,1,0.0
470,470,71.0,24.0,22.245000,12.045000,32.051282,12.341250,1,0.0,70.0,31.0,30.936364,8.811818,34.065934,8.708182,1,0.0
471,471,58.0,15.0,23.447500,11.688750,27.450980,12.028750,1,0.0,37.0,15.0,23.755714,12.698571,24.590164,13.407143,1,0.0
472,472,29.0,8.0,21.275714,12.170000,39.215686,10.874286,1,1.0,62.0,10.0,38.804286,11.938571,64.444444,12.605714,0,1.0


In [14]:
from pycaret.utils import check_metric
check_metric(df1['target'], pred['target'], metric = 'Accuracy')

1.0

In [15]:
####df2

In [16]:
SEED = 0
rg = setup(data = df2, target = 'target', session_id = SEED)

,Description,Value
0,session_id,0
1,Target,target
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(474, 26)"
5,Missing Values,False
6,Numeric Features,23
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


In [17]:
top5 = compare_models(sort='Accuracy', n_select=5 )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.9578,0.0000,0.9445,0.9698,0.9562,0.9154,0.9170,0.013
lda,Linear Discriminant Analysis,0.9517,0.9875,0.9382,0.9635,0.9499,0.9033,0.9049,0.015
lr,Logistic Regression,0.9396,0.9864,0.9379,0.9424,0.9378,0.8790,0.8828,0.177
qda,Quadratic Discriminant Analysis,0.9065,0.9567,0.8890,0.9224,0.9033,0.8126,0.8163,0.014
lightgbm,Light Gradient Boosting Machine,0.9063,0.9733,0.9070,0.9090,0.9062,0.8124,0.8156,0.047
gbc,Gradient Boosting Classifier,0.8947,0.9665,0.8835,0.9044,0.8912,0.7891,0.7934,0.171
knn,K Neighbors Classifier,0.8761,0.9306,0.8463,0.9056,0.8707,0.7523,0.7597,0.114
nb,Naive Bayes,0.8760,0.9441,0.8636,0.8849,0.8718,0.7514,0.7554,0.014
svm,SVM - Linear Kernel,0.8758,0.0000,0.9132,0.8726,0.8806,0.7527,0.7752,0.016
rf,Random Forest Classifier,0.8640,0.9484,0.8327,0.8855,0.8559,0.7273,0.7318,0.503


In [18]:
blended = blend_models(estimator_list=top5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9118,0.0,0.8824,0.9375,0.9091,0.8235,0.8250
1,0.9697,0.0,1.0000,0.9444,0.9714,0.9392,0.9410
2,0.9697,0.0,0.9375,1.0000,0.9677,0.9392,0.9410
3,0.9394,0.0,0.9375,0.9375,0.9375,0.8787,0.8787
4,0.9697,0.0,0.9375,1.0000,0.9677,0.9392,0.9410
5,0.9091,0.0,0.9375,0.8824,0.9091,0.8183,0.8199
6,0.9091,0.0,0.8750,0.9333,0.9032,0.8177,0.8192
7,1.0000,0.0,1.0000,1.0000,1.0000,1.0000,1.0000
8,0.9697,0.0,0.9375,1.0000,0.9677,0.9392,0.9410


In [19]:
mymodels = tune_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9706,0.0,1.0000,0.9444,0.9714,0.9412,0.9428
1,0.9697,0.0,1.0000,0.9444,0.9714,0.9392,0.9410
2,1.0000,0.0,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.9091,0.0,0.8750,0.9333,0.9032,0.8177,0.8192
4,0.9697,0.0,0.9375,1.0000,0.9677,0.9392,0.9410
5,0.8788,0.0,0.8750,0.8750,0.8750,0.7574,0.7574
6,0.9394,0.0,0.8750,1.0000,0.9333,0.8782,0.8848
7,0.9697,0.0,0.9375,1.0000,0.9677,0.9392,0.9410
8,0.9697,0.0,0.9375,1.0000,0.9677,0.9392,0.9410


In [20]:
final_model = finalize_model(mymodels)

In [21]:
pred = predict_model(final_model, data=df2)
pred

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9916,0.9916,0.9913,0.9913,0.9913,0.9831,0.9831


,Unnamed: 0,A팀_득점,A팀_범실,A팀_오픈_성공률,A팀_속공_성공률,A팀_퀵오픈_성공률,A팀_서브_성공률,A팀_블로킹_성공률,A팀_디그_성공률,A팀_디그_점유율,...,B팀_속공_성공률,B팀_퀵오픈_성공률,B팀_서브_성공률,B팀_블로킹_성공률,B팀_디그_성공률,B팀_디그_점유율,B팀_리시브_성공률,B팀_리시브_점유율,B팀MVP,Label
0,0,69.0,25.0,9.437500,2.083750,26.931250,0.175000,11.702128,85.714286,12.498750,...,5.627273,15.150909,0.127273,20.000000,86.111111,9.029091,50.588235,9.090909,1,1.0
1,1,78.0,13.0,35.226667,2.222222,12.036667,0.122222,9.090909,81.250000,10.957778,...,13.226667,13.457778,0.111111,19.047619,88.636364,11.112222,45.192308,11.111111,1,1.0
2,2,88.0,25.0,22.803636,15.426364,10.505455,0.163636,9.722222,83.928571,9.010000,...,14.047000,9.684000,0.140000,11.842105,80.357143,10.001000,38.775510,9.998000,0,0.0
3,3,80.0,19.0,32.128889,22.222222,16.168889,0.155556,10.447761,84.126984,10.852222,...,16.032222,11.111111,0.066667,14.141414,87.272727,11.111111,43.010753,11.112222,1,1.0
4,4,68.0,21.0,19.438750,12.500000,19.633750,0.075000,11.363636,88.118812,12.377500,...,5.000000,8.718000,0.173000,17.741935,88.073394,9.643000,40.000000,9.238000,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,469,46.0,10.0,22.714000,5.000000,32.452000,0.000000,9.375000,87.500000,17.500000,...,20.833750,30.833750,0.082500,7.692308,68.965517,12.286250,31.250000,12.500000,1,0.0
470,470,71.0,24.0,20.486250,6.945000,13.091250,0.150000,19.178082,82.075472,12.045000,...,9.090909,11.620000,0.036364,16.216216,81.052632,8.811818,34.065934,8.708182,1,0.0
471,471,58.0,15.0,12.531250,6.250000,20.441250,0.165000,13.333333,87.500000,11.688750,...,10.714286,20.271429,0.142857,6.382979,79.687500,12.698571,24.590164,13.407143,1,0.0
472,472,29.0,8.0,29.907143,0.000000,8.571429,0.047143,9.302326,81.159420,12.170000,...,20.000000,26.957143,0.188571,16.129032,91.803279,11.938571,64.444444,12.605714,0,1.0


In [22]:
from pycaret.utils import check_metric
check_metric(df2['target'], pred['target'], metric = 'Accuracy')

1.0